# TITLE

## Introduction

A research group in Computer Science at UBC, led by Frank Wood, has set up a Minecraft server to explore how people play and develop interest in video games. However, to fully understand the financial and technical needs of this project, the researchers must know which kinds of players are likely to join the server and how many resources these users will occupy. The following analysis aims to answer the research group’s broad question of “What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?” More specifically, our analysis explores if number of sessions, total number of played hours, and mean session duration can be used to predict if players will subscribe to a game-related newsletter as well as the potential variations between experience level.

### Question 
#### Broad Question: 
Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
<br>
<br>
#### Specific Question:
Can hours played, age, and average session length predict whether a user is subscribed to a game-related newsletter? Additionally does this differ by experience?

### Data
To analyze our question, we will use the following two data sets: 
<br>
<br>


`players.csv` contains information about the users of the Minecraft server, PLAICraft. There are 196 observations and 7 variables as follows: 
- `experience` - character: player's self-determined experience level (`Beginner`, `Amateur`, `Regular`, `Veteran`, or `Pro`)
- `subscribe` - logical: player's subscription status to a game-related newsletter (`TRUE` or `FALSE`)
- `hashedEmail` - character: hashed player's email
- `played_hours` - double: total hours played by each user 
- `name` - character: player's first name
- `gender` - character: player's gender
- `Age` - double: player's age
<br>
<br>

`sessions.csv` contains information about sessions played on PLAICraft. There are 1535 observations and 5 variables as follows:
- `hashedEmail` - character: hashed player's email
- `start_time` - character: session start time in dd/mm/yyyy time 
- `end_time` - character: session end time in dd/mm/yyyy time 
- `orginal_start_time` - double: session start time in milliseconds since January 1st, 1970 at Coordinated Universal Time (UNIX time)
- `orginal_end_time` - double: session end time in milliseconds since January 1st, 1970 at Coordinated Universal Time (UNIX time)

## Methods and Results 

The code below loads the necessary libraries and use the option function to set the viewing of data frames to the first 10 rows. Using a github repository, we will load the players and sessions data to our project.

In [23]:
library(tidyverse)
library(repr)
library(tidymodels)
library(lubridate)
library(ggplot2)
options(repr.matrix.max.rows = 10) 

players_data <- read_csv("https://raw.githubusercontent.com/skylv777/dsci100_group_project/refs/heads/main/players.csv")
head(players.csv)

sessions_data <- read_csv("https://raw.githubusercontent.com/skylv777/dsci100_group_project/refs/heads/main/sessions.csv")
head(sessions.csv)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


The code below alters the sessions data frame variables start_time and end_time to be the type datetime(dttm). This was done to tidy the data and so that the times could be subtracted to create the new column session_length, as in the next steps of creating the combined_data data frame by email to create a big data set. 

In [13]:
#Tidying sessions_data so there is only one value per cell by converting to dttm format
sessions_data_tidy <- sessions_data |>
         mutate(start_time = dmy_hm(start_time)) |>
         mutate(end_time = dmy_hm(end_time))
#Creating sessions_length column
sessions_data_difference <- sessions_data_tidy |>
        mutate(session_length = end_time - start_time) |>
        mutate(session_length = as.double(session_length))
# Determining average session length per player
average_sessions_data <- sessions_data_tidy |>
         mutate(session_length = end_time - start_time) |>
         mutate(session_length = as.double(session_length)) |>
         group_by(hashedEmail) |>
         select(session_length) |>
         summarize(average_session_length = mean(session_length))
# Combining Data Sets
combined_data <- merge(players_data, average_sessions_data)
head(combined_data)

hashedEmail,start_time,end_time,original_start_time,original_end_time,session_length
<chr>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-30 18:12:00,2024-06-30 18:24:00,1.71977e+12,1.71977e+12,12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-06-17 23:33:00,2024-06-17 23:46:00,1.71867e+12,1.71867e+12,13
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,2024-07-25 17:34:00,2024-07-25 17:57:00,1.72193e+12,1.72193e+12,23
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-07-25 03:22:00,2024-07-25 03:58:00,1.72188e+12,1.72188e+12,36
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-05-25 16:01:00,2024-05-25 16:12:00,1.71665e+12,1.71665e+12,11
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-23 15:08:00,2024-06-23 17:10:00,1.71916e+12,1.71916e+12,122


In [24]:
#Summary Statistics on Quantitative Values of players_data 
combined_data_select <- select(combined_data, Age, played_hours, average_session_length)

players_data_stats <- 
        bind_rows(map_df(combined_data_select, mean, na.rm = TRUE), 
                  map_df(combined_data_select, median, na.rm = TRUE),
                  map_df(combined_data_select, ~{
    x <- na.omit(.x)
    if (length(x) == 0) return(NA_real_)
    tibble(val = x) |>
      count(val, sort = TRUE) |>
      filter(n == max(n)) |>
      slice_head(n = 1) |>   
      pull(val)}),
                  map_df(combined_data_select, min, na.rm = TRUE),
                  map_df(combined_data_select, max, na.rm = TRUE), 
                  map_df(combined_data_select, sd, na.rm = TRUE), #?seems too high
                  map_df(combined_data_select, ~ quantile(.x, probs = 0.25, na.rm = TRUE)[[1]]),
                  map_df(combined_data_select, ~ quantile(.x, probs = 0.5, na.rm = TRUE)[[1]]),
                  map_df(combined_data_select, ~ quantile(.x, probs = 0.75, na.rm = TRUE)[[1]])) |>
        mutate(Summary = c("Mean", "Median", "Mode", "Minimum", "Maximum", "Standard Deviation", "1st Quartile", "2nd Quartile", "3rd Quartile")) |>
        relocate(Summary) |>
        mutate(across(Age:average_session_length, \(x) round(x, digits = 2)))
        
players_data_stats

Summary,Age,played_hours,average_session_length
<chr>,<dbl>,<dbl>,<dbl>
Mean,21.59,9.17,31.73
Median,19.00,0.30,18.50
Mode,17.00,0.10,5.00
Minimum,9.00,0.00,5.00
Maximum,58.00,223.10,147.00
Standard Deviation,8.41,35.13,30.86
1st Quartile,17.00,0.10,10.00
2nd Quartile,19.00,0.30,18.50
3rd Quartile,23.00,1.50,43.00


In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)
hours_age_subscribe_plot <- players_data |>
        ggplot(aes(x = Age, y = played_hours, colour = subscribe)) +
        geom_point() +
        labs (x = "Age",
              y = "Total hours played",
              colour = "Subscription Status to Game-Related Newsletter") +
        ggtitle("Total hours played versus Age")
hours_age_subscribe_plot   

It appears that most players are under 30 and have total hours played under 10 hours especially over 30. Additionally, all the players who played more than 20 hours total were subscribed to a newsletter. It appears that users over 30 are less likely to be subscribed to a newsletter.

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)
hours_age_experience_plot <- players_data |>
        ggplot(aes(x = Age, y = played_hours, colour = experience)) +
        geom_point() +
        labs (x = "Age",
              y = "Total hours played",
              colour = "Experience") +
        ggtitle("Total hours played versus Age")
hours_age_experience_plot  

No clear relationship between age and total hours played or experience. However, all the users with over approximately 50 total hours played are either amateur or regular.

In [ ]:
average_age_plot <- combined_data |>
        ggplot(aes(x = Age, y = average_session_length, colour = subscribe)) +
        geom_point() +
        labs (x = "Age",
              y = "Average Session Length (Minutes)",
              colour = "Subscription Status to Game-Related Newsletter") +
        ggtitle("Average Session Length versus Age")
average_age_plot  

It appears that the ratio between users subscribed to users not subscribed to a newsletter is higher with players with average session length above 25 minutes. Additionally, users above the age of 30 appear more likely to have average session length below 25 minutes.

In [ ]:
average_age_experience_plot <- combined_data |>
        ggplot(aes(x = Age, y = average_session_length, colour = experience)) +
        geom_point() +
        labs (x = "Age",
              y = "Average Session Length (Minutes)",
              colour = "Experience") +
        ggtitle("Average Session Length versus Age")
average_age_experience_plot  

## Discussion 